In [30]:
from openai import OpenAI
# openai sdk 是大模型的接口标准，completion/chat
client = OpenAI(
    api_key = 'sk-4589ae8d2971459c9cdb9ad4d9ee1595',
    base_url = 'https://api.deepseek.com/v1'
)

In [31]:
# coding
import sqlite3
# 连接数据库
conn = sqlite3.connect("ai2sql.db")
# 创建一个游标对象，用于执行sql，获取查询结果
cursor = conn.cursor()
# 员工表
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

# list 列表
sample_data = [ 
    # 元组（tuple） 是一种有序的、不可变的序列类型
    (6, "黄佳", "销售", 50000), 
    (7, "宁宁", "工程", 75000), 
    (8, "谦谦", "销售", 60000), 
    (9, "悦悦", "工程", 80000), 
    (10, "黄仁勋", "市场", 55000),
    (11, "曾繁华", "工程", 80000)
]

# 插入sql数据
cursor.executemany(
    "INSERT INTO employees VALUES(?,?,?,?)",
    sample_data
)

# 提交sql事务
conn.commit()

OperationalError: database is locked

In [14]:
# text2sql 给大模型提供什么？上下文context
# 大模型会sql，gemini 博士级别
# 表结构？作为上下文交给prompt，schema 表结构
# PRAGMA 执行语句，table_info 内置函数
# 执行 PRAGMA table_info(employees) 查询 employees 表的结构信息，
# 并使用 .fetchall() 获取所有结果（每行代表一个字段的元数据）
# 返回结果是一个列表，每个元素是一个元组，格式为：
# (cid, name, type, notnull, dflt_value, pk)
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
print(schema)
# 列表推导式
# 构造一个近似 CREATE TABLE 语句的字符串（仅包含字段名和类型，不包含约束等细节）
# 步骤分解：
# 1. 使用列表推导式遍历 schema 中的每一列（col）
#    - col[1] 是字段名（如 'id'）
#    - col[2] 是字段的数据类型（如 'INTEGER'）
#    - 拼接成 "字段名 类型" 的形式，例如 "id INTEGER"
# 2. 用换行符 "\n" 将所有字段定义连接起来
# 3. 在开头加上 "CREATE TABLE EMPLOYEES(\n"，结尾加上 "\n"
schema_str = "CREATE TABLE EMPLOYEES(\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

[(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0), (2, 'department', 'TEXT', 0, None, 0), (3, 'salary', 'INTEGER', 0, None, 0)]
CREATE TABLE EMPLOYEES(
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [21]:
def ask_deepseek(query, schema):
    prompt = f"""
    这是一个数据库的Schema:
    {schema}
    根据这个Schema,请输出一个SQL查询来回答以下问题。
    只输出SQL查询语句本身，不要使用任何Markdown格式，
    不要包含反引号、代码块标记或额外说明。
    问题：{query}
    """
    print(prompt)
    response = client.chat.completions.create(
        model="deepseek-chat",
        max_tokens=2048,
        messages=[{
            "role": "user",
            "content": prompt
        }],
        temperature=0
    )
    return response.choices[0].message.content

In [22]:
question = "工程部门员工的姓名和工资是多少"
sql_query = ask_deepseek(question, schema_str)
print(sql_query)



    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES(
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个Schema,请输出一个SQL查询来回答以下问题。
    只输出SQL查询语句本身，不要使用任何Markdown格式，
    不要包含反引号、代码块标记或额外说明。
    问题：工程部门员工的姓名和工资是多少
    
SELECT name, salary FROM EMPLOYEES WHERE department = '工程';


In [23]:
# ai 生成 sql 能力嵌入开发流程中
results = cursor.execute(sql_query).fetchall()
print(results)

[('宁宁', 75000), ('悦悦', 80000), ('曾繁华', 80000)]


In [ ]:
question = "在销售部门增加一个新员工，姓名为张三，工资为45000"
sql_question = ask_deepseek(question, schema_str)
print(sql_question)


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES(
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个Schema,请输出一个SQL查询来回答以下问题。
    只输出SQL查询语句本身，不要使用任何Markdown格式，
    不要包含反引号、代码块标记或额外说明。
    问题：在销售部门增加一个新员工，姓名为张三，工资为45000
    
INSERT INTO EMPLOYEES (name, department, salary) VALUES ('张三', '销售', 45000);


In [27]:
results = cursor.execute(sql_query)
conn.commit()

In [ ]:
# 大模型加持的后台管理系统
# 数据库平权，小编，业务，任何人加入后台运营和维护

In [33]:
question = "删除市场部门的黄仁勋"
sql_query = ask_deepseek(question, schema_str)
print(sql_query)
cursor.execute(sql_query)
conn.commit()


    这是一个数据库的Schema:
    CREATE TABLE EMPLOYEES(
id INTEGER
name TEXT
department TEXT
salary INTEGER
)
    根据这个Schema,请输出一个SQL查询来回答以下问题。
    只输出SQL查询语句本身，不要使用任何Markdown格式，
    不要包含反引号、代码块标记或额外说明。
    问题：删除市场部门的黄仁勋
    
DELETE FROM EMPLOYEES WHERE department = '市场' AND name = '黄仁勋';


OperationalError: database is locked

In [ ]:
question = "查询所有员工的信息"
sql_query = ask_deepseek(question, schema_str)
print(sql_query)
results = cursor.execute(sql_query).fetchall
